# Data Analysis

**Notes**:
- Only the owner of the account (on disposition) can issue permanent orders and ask for a loan

## Loading the Data

![Data description](./images/data_description.gif)

In [1]:
import pandas as pd
from IPython.display import display

#Reading CSVs and renaming repeated columns

account = pd.read_csv("data/account.csv", sep=";")
client = pd.read_csv("data/client.csv", sep=";")
disposition = pd.read_csv("data/disp.csv", sep=";")
district = pd.read_csv("data/district.csv", sep=";") # Demograph

card_train = pd.read_csv("data/card_train.csv", sep=";")
card_test = pd.read_csv("data/card_test.csv", sep=";")  # Credit Card

loan_train = pd.read_csv("data/loan_train.csv", sep=";")
loan_test = pd.read_csv("data/loan_test.csv", sep=";")

trans_train = pd.read_csv("data/trans_train.csv", sep=";")
trans_test = pd.read_csv("data/trans_test.csv", sep=";")

/home/dani/miniconda3/envs/ac/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


## First Analysis
For a first view of the data, we can check it's shape, some values and data types
### Shape

In [2]:
print(f'''
account shape: {account.shape}
client shape: {client.shape}
disposition shape: {disposition.shape}
district shape: {district.shape}

card_train shape: {card_train.shape}
card_test shape: {card_test.shape}

loan_train shape: {loan_train.shape}
loan_test shape: {loan_test.shape}

trans_train shape: {trans_train.shape}
trans_test shape: {trans_test.shape}
''')


account shape: (4500, 4)
client shape: (5369, 3)
disposition shape: (5369, 4)
district shape: (77, 16)

card_train shape: (177, 4)
card_test shape: (25, 4)

loan_train shape: (328, 7)
loan_test shape: (354, 7)

trans_train shape: (396685, 10)
trans_test shape: (30200, 10)



- Table with most columns: district
- Table with most records: transaction test
- Table with least columns: client
- Table with least records: card test
- 5369 Clients and only 202 credit cards. Not every client has a credit card?
- 5369 Clients and 4500 accounts. Are there clients that do not have an account?
- 5369 Clients and 4500 accounts. Only 682 loans?

### Head
We can check the first 5 entries of each table to get a sense of the data

In [3]:
display("account", account.head())
display("client", client.head())
display("disposition", disposition.head())
display("district", district.head())

display("card_train", card_train.head())
display("card_test", card_test.head())

display("loan_train", loan_train.head())
display("loan_test", loan_test.head())

display("trans_train", trans_train.head())
display("trans_test", trans_test.head())

'account'

,account_id,district_id,frequency,date
0,576,55,monthly issuance,930101
1,3818,74,monthly issuance,930101
2,704,55,monthly issuance,930101
3,2378,16,monthly issuance,930101
4,2632,24,monthly issuance,930102


'client'

,client_id,birth_number,district_id
0,1,706213,18
1,2,450204,1
2,3,406009,1
3,4,561201,5
4,5,605703,5


'disposition'

...

,loan_id,account_id,date,amount,duration,payments,status
0,5895,4473,970103,93960,60,1566,NaN
1,7122,10365,970104,260640,36,7240,NaN
2,6173,5724,970108,232560,48,4845,NaN
3,6142,5591,970121,221880,60,3698,NaN
4,5358,2018,970121,38520,12,3210,NaN


'trans_train'

,trans_id,account_id,date,type,operation,amount,balance,k_symbol,bank,account
0,1548749,5270,930113,credit,credit in cash,800.0,800.0,NaN,NaN,NaN
1,1548750,5270,930114,credit,collection from another bank,44749.0,45549.0,NaN,IJ,80269753.0
2,3393738,11265,930114,credit,credit in cash,1000.0,1000.0,NaN,NaN,NaN
3,3122924,10364,930117,credit,credit in cash,1100.0,1100.0,NaN,NaN,NaN
4,1121963,3834,930119,credit,credit in cash,700.0,700.0,NaN,NaN,NaN


'trans_test'

,trans_id,account_id,date,type,operation,amount,balance,k_symbol,bank,account
0,6145,25,960728,credit,credit in cash,900.0,900.0,NaN,NaN,NaN
1,6456,25,960827,credit,credit in cash,15800.0,16700.0,NaN,NaN,NaN
2,6150,25,960903,credit,credit in cash,13067.0,29767.0,NaN,NaN,NaN
3,6171,25,960905,credit,credit in cash,42054.0,71821.0,NaN,NaN,NaN
4,6457,25,960906,withdrawal,withdrawal in cash,36000.0,77580.0,NaN,NaN,NaN


- Account Id is not ordered on account table
- dates in card, loan and transaction train/test are in a special format (xxyyzz, xx-year, yy-month, zz-day)
- There are some NaN in the data

### Data Types

In [4]:
print('---- account  ----')
account.info()
print('\n---- client ----')
client.info()
print('\n---- disposition ----')
disposition.info()
print('\n---- district ----')
district.info()

print('\n---- card_train ----')
card_train.info()
print('\n---- card_test ----')
card_test.info()

print('\n---- loan_train ----')
loan_train.info()
print('\n---- loan_test ----')
loan_test.info()

print('\n---- trans_train ----')
trans_train.info()
print('\n---- trans_test ----')
trans_test.info()

---- account  ----
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4500 entries, 0 to 4499
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   account_id   4500 non-null   int64 
 1   district_id  4500 non-null   int64 
 2   frequency    4500 non-null   object
 3   date         4500 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 140.8+ KB

---- client ----
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5369 entries, 0 to 5368
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   client_id     5369 non-null   int64
 1   birth_number  5369 non-null   int64
 2   district_id   5369 non-null   int64
dtypes: int64(3)
memory usage: 126.0 KB

---- disposition ----
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5369 entries, 0 to 5368
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 

- There are a lot of datatype of type `object` which could be strings or may have NaN values
- unemploymant rate '95 and no. of commited crimes '95 should be of type float64, but are objects

### Describe Tables

Object types will have values in count, unique, top and freq. While other data types will have values in count, mean, std, min, 25%, 50%, 75% and max. If there are NaN values, is because it it a metric that does not apply to the data type

In [5]:
display("account", account.describe(include='all'))
display("client", client.describe(include='all'))
display("disposition", disposition.describe(include='all'))
display("district", district.describe(include='all'))

display("card_train", card_train.describe(include='all'))
display("card_test", card_test.describe(include='all'))

display("loan_train", loan_train.describe(include='all'))
display("loan_test", loan_test.describe(include='all'))

display("trans_train", trans_train.describe(include='all'))
display("trans_test", trans_test.describe(include='all'))

'account'

,account_id,district_id,frequency,date
count,4500.000000,4500.000000,4500,4500.000000
unique,NaN,NaN,3,NaN
top,NaN,NaN,monthly issuance,NaN
freq,NaN,NaN,4167,NaN
mean,2786.067556,37.310444,NaN,951654.608667
std,2313.811984,25.177217,NaN,14842.188377
min,1.000000,1.000000,NaN,930101.000000
25%,1182.750000,13.000000,NaN,931227.000000
50%,2368.000000,38.000000,NaN,960102.000000
75%,3552.250000,60.000000,NaN,961101.000000


'client'

,client_id,birth_number,district_id
count,5369.000000,5369.000000,5369.000000
mean,3359.011920,535114.970013,37.310114
std,2832.911984,172895.618429,25.043690
min,1.000000,110820.000000,1.000000
25%,1418.000000,406009.000000,14.000000
50%,2839.000000,540829.000000,38.000000
75%,4257.000000,681013.000000,60.000000
max,13998.000000,875927.000000,77.000000


'disposition'

...

,loan_id,account_id,date,amount,duration,payments,status
count,354.000000,354.000000,354.000000,354.000000,354.000000,354.000000,0.0
mean,6141.711864,5677.838983,975109.045198,157063.593220,37.084746,4227.477401,NaN
std,695.356054,3345.166236,4928.280822,120285.252766,17.387392,2238.683851,NaN
min,4962.000000,25.000000,970103.000000,5148.000000,12.000000,304.000000,NaN
25%,5528.000000,2734.250000,970619.000000,66696.000000,24.000000,2486.750000,NaN
50%,6100.500000,5378.500000,971208.000000,120804.000000,36.000000,4003.000000,NaN
75%,6761.750000,8764.500000,980516.500000,220437.000000,48.000000,5686.500000,NaN
max,7295.000000,11328.000000,981208.000000,590820.000000,60.000000,9910.000000,NaN


'trans_train'

,trans_id,account_id,date,type,operation,amount,balance,k_symbol,bank,account
count,3.966850e+05,396685.000000,396685.000000,396685,325924,396685.000000,396685.000000,211441,97242,1.022290e+05
unique,NaN,NaN,NaN,3,5,NaN,NaN,7,13,NaN
top,NaN,NaN,NaN,withdrawal,withdrawal in cash,NaN,NaN,interest credited,ST,NaN
freq,NaN,NaN,NaN,232093,165270,NaN,NaN,70761,8114,NaN
mean,1.239338e+06,2508.434796,951310.066801,NaN,NaN,5677.552980,35804.792507,NaN,NaN,4.664229e+07
std,1.213288e+06,2020.928889,9510.974536,NaN,NaN,9190.364137,19692.148243,NaN,NaN,3.002136e+07
min,1.000000e+00,1.000000,930101.000000,NaN,NaN,0.000000,-13588.700000,NaN,NaN,0.000000e+00
25%,3.918330e+05,1092.000000,941110.000000,NaN,NaN,127.500000,22424.300000,NaN,NaN,1.990018e+07
50%,7.882580e+05,2220.000000,950930.000000,NaN,NaN,1952.000000,30959.600000,NaN,NaN,4.673618e+07
75%,1.273700e+06,3357.000000,960606.000000,NaN,NaN,6500.000000,44661.000000,NaN,NaN,7.232217e+07


'trans_test'

,trans_id,account_id,date,type,operation,amount,balance,k_symbol,bank,account
count,3.020000e+04,30200.000000,30200.000000,30200,25070,30200.000000,30200.000000,12781,5823,9.139000e+03
unique,NaN,NaN,NaN,3,5,NaN,NaN,6,13,NaN
top,NaN,NaN,NaN,withdrawal,withdrawal in cash,NaN,NaN,interest credited,YZ,NaN
freq,NaN,NaN,NaN,17462,13889,NaN,NaN,5130,501,NaN
mean,1.997768e+06,5639.798344,969666.689901,NaN,NaN,8978.861517,44644.210795,NaN,NaN,3.352367e+07
std,1.172841e+06,3362.670775,6984.081578,NaN,NaN,12456.202612,24068.542647,NaN,NaN,3.438393e+07
min,6.145000e+03,25.000000,950419.000000,NaN,NaN,0.200000,-17030.400000,NaN,NaN,0.000000e+00
25%,9.273805e+05,2720.000000,961214.000000,NaN,NaN,216.800000,27441.450000,NaN,NaN,0.000000e+00
50%,2.042316e+06,5477.000000,970522.000000,NaN,NaN,3700.000000,40378.200000,NaN,NaN,2.314050e+07
75%,3.117521e+06,8772.000000,971117.000000,NaN,NaN,12300.000000,57737.200000,NaN,NaN,6.515492e+07


## Explore the Dataset

First thing to analyse are the data type `object` columns

In [30]:
print("------ account ------")
print("-> frequency")
display(account["frequency"].value_counts())

print("\n------ disposition ------")
print("-> type")
display(disposition["type"].value_counts())

print("\n------ district ------")
print("-> name")
display(district["name "].value_counts())
print("-> region")
display(district["region"].value_counts())
print("-> unemploymant rate '95")
display(district["unemploymant rate '95 "].value_counts())
print("-> no. of commited crimes '95")
display(district["no. of commited crimes '95 "].value_counts())


print("\n------ card_train ------")
print("-> type")
display(card_train["type"].value_counts())

print("\n------ card_test ------")
print("-> type")
display(card_test["type"].value_counts())

print("\n------ trans_train ------")
print("-> type")
display(trans_train["type"].value_counts())
print("-> operation")
display(trans_train["operation"].value_counts())
print("-> k_symbol")
display(trans_train["k_symbol"].value_counts())
print("-> bank")
display(trans_train["bank"].value_counts())

print("\n------ trans_test ------")
print("-> type")
display(trans_test["type"].value_counts())
print("-> operation")
display(trans_test["operation"].value_counts())
print("-> k_symbol")
display(trans_test["k_symbol"].value_counts())
print("-> bank")
display(trans_test["bank"].value_counts())

------ account ------
-> frequency


monthly issuance              4167
weekly issuance                240
issuance after transaction      93
Name: frequency, dtype: int64


------ disposition ------
-> type


OWNER        4500
DISPONENT     869
Name: type, dtype: int64


------ district ------
-> name


...

withdrawal in cash              13889
credit in cash                   5304
remittance to another bank       4375
collection from another bank     1448
credit card withdrawal             54
Name: operation, dtype: int64

-> k_symbol


interest credited                        5130
payment for statement                    3186
household                                2192
                                         1549
insurrance payment                        681
sanction interest if negative balance      43
Name: k_symbol, dtype: int64

-> bank


YZ    501
UV    500
WX    497
KL    482
QR    460
OP    446
AB    438
IJ    438
GH    432
MN    431
CD    414
EF    401
ST    383
Name: bank, dtype: int64

- Some columns have a ' ' after the column name in district
- Transaction types (+/-) should be only withdrawal and credit, withdrawal in cash shouldn't be there (should be only withdrawal)
- k-symbol as white values
